In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Sample Dataset Preparation
class AdDataset(Dataset):
    def __init__(self, df, user_features, ad_features, dense_features, label):
        self.df = df
        self.user_features = user_features
        self.ad_features = ad_features
        self.dense_features = dense_features
        self.label = label
        
        # Label encoding for categorical features
        self.label_encoders = {col: LabelEncoder().fit(self.df[col]) for col in user_features + ad_features}
        for col, le in self.label_encoders.items():
            self.df[col] = le.transform(self.df[col])
        
        # MinMax scaling for dense features
        self.scaler = MinMaxScaler().fit(self.df[dense_features])
        self.df[dense_features] = self.scaler.transform(self.df[dense_features])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        user = torch.tensor(row[self.user_features].values, dtype=torch.long)
        ad = torch.tensor(row[self.ad_features].values, dtype=torch.long)
        dense = torch.tensor(row[self.dense_features].values, dtype=torch.float)
        label = torch.tensor(row[self.label], dtype=torch.float)
        return user, ad, dense, label

# DeepFM Model
class DeepFM(nn.Module):
    def __init__(self, user_feat_dims, ad_feat_dims, dense_feat_dim, embedding_dim, hidden_units):
        super(DeepFM, self).__init__()
        self.embedding_dim = embedding_dim
        
        # Embedding layers for user and ad features
        self.user_embeddings = nn.ModuleList([nn.Embedding(feat_dim, embedding_dim) for feat_dim in user_feat_dims])
        #print('0', self.user_embeddings)
        self.ad_embeddings = nn.ModuleList([nn.Embedding(feat_dim, embedding_dim) for feat_dim in ad_feat_dims])
        #print('1', self.ad_embeddings)
        # Dense feature layers
        self.dense_linear = nn.Linear(dense_feat_dim, 1)
        #print('2', self.dense_linear)
        
        # FM part: second-order interactions
        self.fm_first_order = nn.ModuleList([nn.Embedding(feat_dim, 1) for feat_dim in user_feat_dims + ad_feat_dims])

        # DNN part: multi-layer perceptron
        input_dim = len(user_feat_dims + ad_feat_dims) * embedding_dim + dense_feat_dim
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_units[0]),
            nn.ReLU(),
            nn.Linear(hidden_units[0], hidden_units[1]),
            nn.ReLU(),
            nn.Linear(hidden_units[1], 1)
        )
        #print('4', self.mlp)
        
        # Output layer
        self.output_layer = nn.Sigmoid()

    def forward(self, user_inputs, ad_inputs, dense_inputs):
        # Embedding lookup for user and ad features
        user_embeds = [emb(user_inputs[:, i]) for i, emb in enumerate(self.user_embeddings)] # (num_features, batch_size, embed_dim)
        ad_embeds = [emb(ad_inputs[:, i]) for i, emb in enumerate(self.ad_embeddings)] # (num_features, batch_size, embed_dim)
        
        # FM first-order term
        first_order_user = [emb(user_inputs[:, i]) for i, emb in enumerate(self.fm_first_order[:len(user_embeds)])]
        first_order_ad = [emb(ad_inputs[:, i]) for i, emb in enumerate(self.fm_first_order[len(user_embeds):])]
        fm_first_order = torch.cat(first_order_user + first_order_ad, dim=1)
        #print('5', fm_first_order)
        
        # FM second-order interactions (factorized part)
        user_embeds = torch.stack(user_embeds, dim=1)  # (batch_size, num_features, embed_dim)
        ad_embeds = torch.stack(ad_embeds, dim=1)
        fm_embeddings = torch.cat([user_embeds, ad_embeds], dim=1)
        fm_sum_square = torch.sum(fm_embeddings, dim=1) ** 2
        fm_square_sum = torch.sum(fm_embeddings ** 2, dim=1)
        fm_second_order = 0.5 * (fm_sum_square - fm_square_sum)
        print(user_embeds.shape, ad_embeds.shape, fm_embeddings.shape, fm_sum_square.shape, fm_square_sum.shape, fm_second_order.shape)
        
        # DNN part
        dnn_input = torch.cat([torch.flatten(user_embeds, start_dim=1), torch.flatten(ad_embeds, start_dim=1), dense_inputs], dim=1)
        print(dnn_input.shape)
        dnn_output = self.mlp(dnn_input)
        print(dnn_output.shape)
        
        # Combine FM and DNN parts
        total_output = fm_first_order.sum(dim=1) + fm_second_order.sum(dim=1) + dnn_output.squeeze(1)
        return self.output_layer(total_output)

# Sample Training Script
def train_model(df, user_features, ad_features, dense_features, label, epochs=5):
    dataset = AdDataset(df, user_features, ad_features, dense_features, label)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    model = DeepFM(user_feat_dims=[df[col].nunique() for col in user_features],
                   ad_feat_dims=[df[col].nunique() for col in ad_features],
                   dense_feat_dim=len(dense_features),
                   embedding_dim=8,
                   hidden_units=[128, 64])
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCELoss()
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for user_inputs, ad_inputs, dense_inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(user_inputs, ad_inputs, dense_inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}")

In [2]:
# Sample data
data = {
    'user_id': [1, 2, 1, 3],
    'ad_id': [101, 102, 103, 101],
    'age': [25, 35, 25, 45],
    'income': [50000, 60000, 50000, 80000],
    'clicked': [1, 0, 1, 0]
}
df = pd.DataFrame(data)

# Defining feature sets
user_features = ['user_id']
ad_features = ['ad_id']
dense_features = ['age', 'income']
label = 'clicked'

In [4]:
# Training the model
train_model(df, user_features, ad_features, dense_features, label, epochs=5)

torch.Size([4, 1, 8]) torch.Size([4, 1, 8]) torch.Size([4, 2, 8]) torch.Size([4, 8]) torch.Size([4, 8]) torch.Size([4, 8])
torch.Size([4, 18])
torch.Size([4, 1])
Epoch 1/5, Loss: 1.8068265914916992
torch.Size([4, 1, 8]) torch.Size([4, 1, 8]) torch.Size([4, 2, 8]) torch.Size([4, 8]) torch.Size([4, 8]) torch.Size([4, 8])
torch.Size([4, 18])
torch.Size([4, 1])
Epoch 2/5, Loss: 1.7710514068603516
torch.Size([4, 1, 8]) torch.Size([4, 1, 8]) torch.Size([4, 2, 8]) torch.Size([4, 8]) torch.Size([4, 8]) torch.Size([4, 8])
torch.Size([4, 18])
torch.Size([4, 1])
Epoch 3/5, Loss: 1.7372453212738037
torch.Size([4, 1, 8]) torch.Size([4, 1, 8]) torch.Size([4, 2, 8]) torch.Size([4, 8]) torch.Size([4, 8]) torch.Size([4, 8])
torch.Size([4, 18])
torch.Size([4, 1])
Epoch 4/5, Loss: 1.704978346824646
torch.Size([4, 1, 8]) torch.Size([4, 1, 8]) torch.Size([4, 2, 8]) torch.Size([4, 8]) torch.Size([4, 8]) torch.Size([4, 8])
torch.Size([4, 18])
torch.Size([4, 1])
Epoch 5/5, Loss: 1.6745109558105469


In [5]:
df[dense_features]

,age,income
0,0.0,0.000000
1,0.5,0.333333
2,0.0,0.000000
3,1.0,1.000000


In [6]:
scaler = MinMaxScaler().fit(df[dense_features])
scaler.transform(df[dense_features])

array([[0.        , 0.        ],
       [0.5       , 0.33333333],
       [0.        , 0.        ],
       [1.        , 1.        ]])

In [7]:
le = LabelEncoder().fit(torch.tensor(df.user_id))
le.transform(df.user_id)

array([0, 1, 0, 2])